In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [35]:
tickers = [
    "NVDA",
    "AMD",
    "TSM",
    "AAPL",
    "MSFT",
    "GOOGL",
    "AMZN",
    "META",
    "TSLA",
    "PLTR",
    "LIDR",
    "OPEN",
    "SOFI",
    "IONQ",
]
start, end = "2025-04-25", "2025-07-25"

# 3‑month Treasury yield on 25 Jul 2025 ≈ 4.42 %
rf_annual = 0.0442
rf_daily = (1 + rf_annual) ** (1 / 252) - 1
rf_monthly = (1 + rf_annual) ** (1 / 12) - 1

In [41]:
def sharpe(series, rf):
    excess = series - rf
    return (excess.mean() / excess.std(ddof=0)).values[0]


rows = []
for t in tickers:
    px = yf.download(t, start=start, end=end, progress=False)["Close"]
    daily_ret = px.pct_change().dropna()
    ann_sharpe = np.sqrt(252) * sharpe(daily_ret, rf_daily)

    monthly_ret = px.resample("M").last().pct_change().dropna()
    mon_sharpe = np.sqrt(12) * sharpe(monthly_ret, rf_monthly)

    rows.append((t, ann_sharpe, mon_sharpe))

/var/folders/28/m7gm_18956vc6yqrg114c9wh0000gn/T/ipykernel_86393/783122381.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(t, start=start, end=end, progress=False)['Close']
/var/folders/28/m7gm_18956vc6yqrg114c9wh0000gn/T/ipykernel_86393/783122381.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ret = px.resample('M').last().pct_change().dropna()
/var/folders/28/m7gm_18956vc6yqrg114c9wh0000gn/T/ipykernel_86393/783122381.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(t, start=start, end=end, progress=False)['Close']
/var/folders/28/m7gm_18956vc6yqrg114c9wh0000gn/T/ipykernel_86393/783122381.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ret = px.resample('M').last().pct_change().dropna()
/var/folders/28/m7gm_18956vc6yqrg114c9wh0000gn/T/ipykernel

In [43]:
out = pd.DataFrame(rows, columns=["Ticker", "Sharpe_Annual", "Sharpe_Monthly"])
print(out.set_index("Ticker").round(2).sort_values("Sharpe_Annual", ascending=False))

        Sharpe_Annual  Sharpe_Monthly
Ticker                               
NVDA             6.15           10.01
MSFT             5.57            5.29
AMD              5.40            9.53
TSM              5.29            9.53
SOFI             4.97            5.52
META             3.83            3.50
OPEN             3.36            2.12
AMZN             3.22           11.62
LIDR             2.64            3.29
PLTR             2.54            7.24
GOOGL            2.50            7.81
IONQ             2.11            3.12
TSLA             0.71            0.80
AAPL             0.33           -0.03
